<a href="https://colab.research.google.com/github/Muppidathe/ml_project/blob/master/h_test%2Canova.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("iabhishekofficial/mobile-price-classification")

print("Path to dataset files:", path)
!mv $path ./dataset

Path to dataset files: /kaggle/input/mobile-price-classification
mv: cannot remove '/kaggle/input/mobile-price-classification/train.csv': Read-only file system
mv: cannot remove '/kaggle/input/mobile-price-classification/test.csv': Read-only file system


In [3]:
import pandas as pd
df=pd.read_csv('dataset/train.csv')
df

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,842,0,2.2,0,1,0,7,0.6,188,2,...,20,756,2549,9,7,19,0,0,1,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,...,905,1988,2631,17,3,7,1,1,0,2
2,563,1,0.5,1,2,1,41,0.9,145,5,...,1263,1716,2603,11,2,9,1,1,0,2
3,615,1,2.5,0,0,0,10,0.8,131,6,...,1216,1786,2769,16,8,11,1,0,0,2
4,1821,1,1.2,0,13,1,44,0.6,141,2,...,1208,1212,1411,8,2,15,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,794,1,0.5,1,0,1,2,0.8,106,6,...,1222,1890,668,13,4,19,1,1,0,0
1996,1965,1,2.6,1,0,0,39,0.2,187,4,...,915,1965,2032,11,10,16,1,1,1,2
1997,1911,0,0.9,1,1,1,36,0.7,108,8,...,868,1632,3057,9,1,5,1,1,0,3
1998,1512,0,0.9,0,4,1,46,0.1,145,5,...,336,670,869,18,10,19,1,1,1,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   battery_power  2000 non-null   int64  
 1   blue           2000 non-null   int64  
 2   clock_speed    2000 non-null   float64
 3   dual_sim       2000 non-null   int64  
 4   fc             2000 non-null   int64  
 5   four_g         2000 non-null   int64  
 6   int_memory     2000 non-null   int64  
 7   m_dep          2000 non-null   float64
 8   mobile_wt      2000 non-null   int64  
 9   n_cores        2000 non-null   int64  
 10  pc             2000 non-null   int64  
 11  px_height      2000 non-null   int64  
 12  px_width       2000 non-null   int64  
 13  ram            2000 non-null   int64  
 14  sc_h           2000 non-null   int64  
 15  sc_w           2000 non-null   int64  
 16  talk_time      2000 non-null   int64  
 17  three_g        2000 non-null   int64  
 18  touch_sc

In [8]:
df.duplicated().sum()

np.int64(0)

In [38]:
from scipy.stats import anderson
from scipy.stats import chi2_contingency,f_oneway,kruskal
importand_cloumn=[]
unwanted_column=[]
for column in df.drop(columns=['price_range']).columns:
  if len(df[column].unique())<=2:
    # print('binary',column)
    contingency = pd.crosstab(df['price_range'], df[column])
    chi2,p,_,_=chi2_contingency(contingency)
    if p<.05:
      print('there are some relationship we can use ',column)
      importand_cloumn.append(column)
    else:
      # print('there are no relationship we can\'t use ',column)
      unwanted_column.append(column)

  elif len(df[column].unique())<=5:
    # print('multiclass',column)
    continue
  else:
    # print('continious',column)
    normality=1
    for classs in df['price_range'].unique():
      new_df=df[df['price_range']==classs][column]
      result=anderson(new_df,dist='norm')
      if result.statistic<result.critical_values[2]:
        # print("accept null hypothesis(normal distribution only)")
        normality=1
      else:
        # print("reject null hypothesis(non-normal distribution)")
        normality=0
        break
    groups = [df[df['price_range'] == cls][column] for cls in df['price_range'].unique()]
    if normality==0:
      stat, p = kruskal(*groups)
      if p>.05:
        # print("→ No significant difference between group means.")
        unwanted_column.append(column)
      else:
        print("→ Significant differences exist between group means.",column)
        importand_cloumn.append(column)
    else:
      stat, p = f_oneway(*groups)
      if p>.05:
        # print("→ No significant difference between group means.")
        unwanted_column.append(column)
      else:
        print("→ Significant differences exist between group means.",column)
        importand_cloumn.append(column)

→ Significant differences exist between group means. battery_power
→ Significant differences exist between group means. int_memory
→ Significant differences exist between group means. mobile_wt
→ Significant differences exist between group means. n_cores
→ Significant differences exist between group means. px_height
→ Significant differences exist between group means. px_width
→ Significant differences exist between group means. ram


In [36]:
importand_cloumn

['battery_power',
 'int_memory',
 'mobile_wt',
 'n_cores',
 'px_height',
 'px_width',
 'ram']

In [39]:
unwanted_column

['blue',
 'clock_speed',
 'dual_sim',
 'fc',
 'four_g',
 'm_dep',
 'pc',
 'sc_h',
 'sc_w',
 'talk_time',
 'three_g',
 'touch_screen',
 'wifi']

In [41]:
new_df=df[importand_cloumn]
x=new_df
y=df['price_range']
new_df

,battery_power,int_memory,mobile_wt,n_cores,px_height,px_width,ram
0,842,7,188,2,20,756,2549
1,1021,53,136,3,905,1988,2631
2,563,41,145,5,1263,1716,2603
3,615,10,131,6,1216,1786,2769
4,1821,44,141,2,1208,1212,1411
...,...,...,...,...,...,...,...
1995,794,2,106,6,1222,1890,668
1996,1965,39,187,4,915,1965,2032
1997,1911,36,108,8,868,1632,3057
1998,1512,46,145,5,336,670,869


In [45]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=.2,random_state=42)

In [42]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler

In [46]:
transformer=ColumnTransformer(transformers=[('transform',StandardScaler(),new_df.columns)])
random=Pipeline([('pre processing',transformer),('model',RandomForestClassifier(random_state=42))])
random.fit(x_train,y_train)

Pipeline(steps=[('pre processing',
                 ColumnTransformer(transformers=[('transform', StandardScaler(),
                                                  Index(['battery_power', 'int_memory', 'mobile_wt', 'n_cores', 'px_height',
       'px_width', 'ram'],
      dtype='object'))])),
                ('model', RandomForestClassifier(random_state=42))])

In [48]:
from sklearn.metrics import f1_score,accuracy_score
train_res=random.predict(x_train)
train_accur=accuracy_score(y_train,train_res)
print('train accuracy',train_accur)



train accuracy 1.0


In [53]:
from sklearn.metrics import f1_score,accuracy_score
test_res=random.predict(x_test)
test_accur=accuracy_score(y_test,test_res)
print('test accuracy',test_accur)

test accuracy 0.925


In [50]:
from sklearn.linear_model import LogisticRegression
logistic=Pipeline([('pre processing',transformer),('model',LogisticRegression())])
logistic.fit(x_train,y_train)

Pipeline(steps=[('pre processing',
                 ColumnTransformer(transformers=[('transform', StandardScaler(),
                                                  Index(['battery_power', 'int_memory', 'mobile_wt', 'n_cores', 'px_height',
       'px_width', 'ram'],
      dtype='object'))])),
                ('model', LogisticRegression())])

In [52]:
from sklearn.metrics import f1_score,accuracy_score
train_res=logistic.predict(x_train)
train_accur=accuracy_score(y_train,train_res)
print('train accuracy',train_accur)

train accuracy 0.974375


In [54]:
from sklearn.metrics import f1_score,accuracy_score
test_res=logistic.predict(x_test)
test_accur=accuracy_score(y_test,test_res)
print('test accuracy',test_accur)

test accuracy 0.9775


In [68]:
from sklearn.svm import SVC
svc=Pipeline([('pre processing',transformer),('model',SVC(kernel='rbf'))])
svc.fit(x_train,y_train)

Pipeline(steps=[('pre processing',
                 ColumnTransformer(transformers=[('transform', StandardScaler(),
                                                  Index(['battery_power', 'int_memory', 'mobile_wt', 'n_cores', 'px_height',
       'px_width', 'ram'],
      dtype='object'))])),
                ('model', SVC())])

In [69]:
from sklearn.metrics import f1_score,accuracy_score
train_res=svc.predict(x_train)
train_accur=accuracy_score(y_train,train_res)
print('train accuracy',train_accur)

train accuracy 0.973125


In [70]:
from sklearn.metrics import f1_score,accuracy_score
test_res=svc.predict(x_test)
test_accur=accuracy_score(y_test,test_res)
print('test accuracy',test_accur)

test accuracy 0.9425


In [55]:
from sklearn.neighbors import KNeighborsClassifier
knn=Pipeline([('pre processing',transformer),('model',KNeighborsClassifier(n_neighbors=13))])
knn.fit(x_train,y_train)

Pipeline(steps=[('pre processing',
                 ColumnTransformer(transformers=[('transform', StandardScaler(),
                                                  Index(['battery_power', 'int_memory', 'mobile_wt', 'n_cores', 'px_height',
       'px_width', 'ram'],
      dtype='object'))])),
                ('model', KNeighborsClassifier(n_neighbors=13))])

In [56]:
from sklearn.metrics import f1_score,accuracy_score
train_res=knn.predict(x_train)
train_accur=accuracy_score(y_train,train_res)
print('train accuracy',train_accur)

train accuracy 0.8375


In [57]:
from sklearn.metrics import f1_score,accuracy_score
test_res=knn.predict(x_test)
test_accur=accuracy_score(y_test,test_res)
print('test accuracy',test_accur)

test accuracy 0.7775


# logistic regression works well with the above dataset with 97% accuracy for both train and test data

